In [ ]:
######################################################
#Name: Shweta Ydaav
#Student ID : 21209251
#Purpose : To acquire input json/csv file from web using api and uploading it to mongodb nosql database using python
#######################################################
import pymongo
import json
import requests
import pandas as pd
import os
from zipfile import ZipFile #importing the zipfile module

#Kaggle credential to read data from kaggle API
kaggleUsername = "panda05"
kaggleKey = "c25dc978d7e3721eccfd1f4814846223"

#Setting the enovironmemt credential variables
os.environ['KAGGLE_USERNAME'] = kaggleUsername
os.environ['KAGGLE_KEY'] = kaggleKey

#Kaggle API call to download the data folder
try:
    !kaggle datasets download -d berkeleyearth/climate-change-earth-surface-temperature-data
except Exception as e:
    print("Data download failed with error :" + str(e))

#Extracting the data file from downloaded zip
with ZipFile("climate-change-earth-surface-temperature-data.zip", 'r') as zObject:
	zObject.extractall(path = "surface-temperature")

#Deleting the zipped folder
os.remove("climate-change-earth-surface-temperature-data.zip")

#Setting port number for localhost
dbPortNumber = '27017'  

#Create a client to connect to server DB
mongoClient = pymongo.MongoClient(f'mongodb://localhost:{dbPortNumber}')

#Connect to database if exists, create db if it doesn't exist.
mongoDB = mongoClient['dapDatabase']

#Create  the tables in the database.
changeInSeaLevel = mongoDB['changeInSeaLevel']
surfaceTemperature = mongoDB['surfaceTemperature']
naturalDisaster = mongoDB['naturalDisaster']

#to drop the db tables
changeInSeaLevel.drop()
surfaceTemperature.drop()
naturalDisaster.drop()

#API call to load data for change in sea level
#The API can only pull 2000 records in one call. Using exceededTransferLimit parameter in api response to loop and pull all 35k records
response = requests.get('https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Indicator_3_3_melted_new/FeatureServer/0/query?where=1%3D1&resultRecordCount=35000&outFields=*&outSR=4326&f=json')
listToWrite = response.json()
count = 0
offset = 0
i=1;
try :
	while (listToWrite['exceededTransferLimit']):
		lisOfRecords = listToWrite['features']
		for item in lisOfRecords:
			attr = item['attributes']
			attr['ObjectId'] = str(attr['ObjectId'])
			attr['Country'] = str(attr['Country'])
			attr['ISO2'] = str(attr['ISO2'])
			attr['ISO3'] = str(attr['ISO3'])
			attr['Indicator'] = str(attr['Indicator'])
			attr['Unit'] = str(attr['Unit'])
			attr['Source'] = str(attr['Source'])
			attr['CTS_Code'] = str(attr['CTS_Code'])
			attr['CTS_Name'] = str(attr['CTS_Name'])
			attr['CTS_Full_Descriptor'] = str(attr['CTS_Full_Descriptor'])
			attr['Measure'] = str(attr['Measure'])
			attr['Date'] = str(attr['Date'])
			attr['Value'] = str(attr['Value'])
			insertProcess = changeInSeaLevel.insert_one(attr)
			count = count+1
		offset = offset + 2000
		response = requests.get('https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Indicator_3_3_melted_new/FeatureServer/0/query?where=1%3D1&resultOffset='+str(offset)+'&outFields=*&outSR=4326&f=json')
		listToWrite = response.json()
		i = i+1
except Exception as e:
	if ("'exceededTransferLimit'" in str(e)):
		lisOfRecords = listToWrite['features']
		for item in lisOfRecords:
			attr = item['attributes']
			attr['ObjectId'] = str(attr['ObjectId'])
			attr['Country'] = str(attr['Country'])
			attr['ISO2'] = str(attr['ISO2'])
			attr['ISO3'] = str(attr['ISO3'])
			attr['Indicator'] = str(attr['Indicator'])
			attr['Unit'] = str(attr['Unit'])
			attr['Source'] = str(attr['Source'])
			attr['CTS_Code'] = str(attr['CTS_Code'])
			attr['CTS_Name'] = str(attr['CTS_Name'])
			attr['CTS_Full_Descriptor'] = str(attr['CTS_Full_Descriptor'])
			attr['Measure'] = str(attr['Measure'])
			attr['Date'] = str(attr['Date'])
			attr['Value'] = str(attr['Value'])
			insertProcess = changeInSeaLevel.insert_one(attr)
			count = count+1

#total number of records loaded in data
print("Data for change in sea level successfully loaded to mongoDB. Records loaded: "+ str(count))


#loading data for surface temperature
try:
	data = pd.read_csv('surface-temperature/GlobalLandTemperaturesByMajorCity.csv')
	docs = json.loads(data.T.to_json()).values()
	surfaceTemperature.insert_many(docs)
	print("Data for surface temperature successfully loaded to mongoDB.")
except Exception as e:
	print('Data load to mongodb failed with error '+ str(e))
	

#loading data for natural disaster
try:
	data = pd.read_csv('1900_2021_DISASTERS.csv')
	docs = json.loads(data.T.to_json()).values()
	naturalDisaster.insert_many(docs)
	print("Data for natural disaster successfully loaded to mongoDB.")
except Exception as e:
	print('Data load to mongodb failed with error '+ str(e))
    
#close mongodb client connection
mongoClient.close()